In [1]:
import pandas as pd 
import numpy as np
import os
from subprocess import check_output
import shutil

from rule import *
from amie import *

In [2]:
root_source = "./../../FB15k-237-num/"
store_data = "./../../Data_FB/FB15K_baseline_no_numeric.tsv"

In [3]:
data = open(root_source+"train.txt", "r")
f = open(store_data, "w")

predicate_to_find = set()

for line in data: 
    f.write(line)
    predicate_to_find.add(line.split("\n")[0].split("\t")[1])
    
data.close()
f.close()

In [4]:
htr = ""
for p in predicate_to_find:
    htr+=p+","
htr = htr[:-1]

In [6]:
res = check_output(f'java -jar ./../amie3.jar -htr {htr} {store_data}', shell=True)

res_parsed = parse_amie(res)

In [7]:
len(res_parsed)

7860

In [8]:
res_parsed

{?b /award/award_nominee/award_nominations./award/award_nomination/nominated_for ?f & ?a /film/film_distributor/films_distributed./film/film_film_distributor_relationship/film ?f => ?a /organization/organization/child./organization/organization_relationship/child ?b,
 ?a /base/schemastaging/person_extra/net_worth./measurement_unit/dated_money_value/currency ?f & ?f /common/topic/webpage./common/webpage/category ?b => ?a /common/topic/webpage./common/webpage/category ?b,
 ?b /base/aareas/schema/administrative_area/administrative_parent ?f & ?f /location/administrative_division/first_level_division_of ?a => ?a /location/location/contains ?b,
 ?a /award/award_nominee/award_nominations./award/award_nomination/nominated_for ?f & ?f /tv/tv_program/regular_cast./tv/regular_tv_appearance/actor ?b => ?a /award/award_nominee/award_nominations./award/award_nomination/award_nominee ?b,
 ?e /location/statistical_region/religions./location/religion_percentage/religion ?b & ?e /military/military_comb